### Comparing the complexity of the encoding methods.

- In this example, various inputs are encoded using the basis, angle, amplitude, and divide-and-conquer encoding methods.

- The resulting quantum circuits are then compared based on their qubit counts and gate counts.

---

Firstly, the `qsp` package that contains the encoding algorithms is imported (it is essential to preserve the same file structure as in the published repository; otherwise, the import may not work):

In [1]:
import os

# Get the directory of this Notebook file:
current_notebook_directory = get_ipython().run_line_magic("pwd", "").strip()

# Change the current working directory to this Notebook's directory:
os.chdir(current_notebook_directory)

# Move up two directories (to the root of the repository):
os.chdir("./../")

# Import the quantum state preparation package:
import qsp

Throughout this whole example, two values are calculated for each quantum circuit:
- `qubit count`, meaning how many qubits a circuit uses to encode an input
- `gate count`, meaning how many gates a circuit uses to encode an input

These two values are then used to compare the encoding methods to each other.

Regarding the `gate count` number, it is important to note that the number depends on which gates are counted. If the gates that are added to a circuit during the encoding process are counted directly, the resulting number may be lower than the actual number of gates used in a real quantum computer. That may happen because some gates added during the encoding process are not supported by the `IBM Quantum` machines, such as `ry` gates or `mcry` gates. So, the quantum circuit needs to be transpiled to get the actual number of gates used in a real quantum computer. However, even among the `IBM Quantum` computers, each might support a different set of gates. Moreover, the transpilation process itself differs based on the chosen optimization level. In this Notebook, three different `gate count` calculations are performed:

1. Directly counting the gates that are added by the implemented encoding methods and not transpiling created circuits.
   
   This first approach just counts how many gates the implemented encoding algorithms use to encode a given input vector.


2. Counting the gates after transpiling the circuits with the lowest possible optimization level.

    This approach is more informative because it takes into account the actual number of gates used when encoding an input in a real quantum computer. During the transpilation process, the lowest possible optimization level is used, meaning that no optimization is performed when transpiling the circuit. All gates are transpiled in the most straightforward way possible, so the `gate count` is highest in this case. This approach tests the worst-case scenario, meaning the resulting `gate count` can only get better regardless of which optimization algorithm is utilized.


3. Counting the gates after transpiling the circuits with the highest possible optimization level.

    This last approach is similar to the second one, but the highest optimization level is chosen when the transpilation is performed. The results can change in the future if the optimization algorithms used in the `Qiskit` library change over time.

When the transpilation is performed, each circuit is transpiled so that it only uses the gates that are supported by the `IBM Quantum` machines. However, each `IBM Quantum` computer might support a different set of gates. In this Notebook, the most recent processor architecture from `IBM` is used as a target to get the list of supported gates. As of May 2024, the `IBM Heron` processor family is the latest, most advanced processor family, so the transpilation process performed in this Notebook only uses gates that are supported by this processor.

As of May 2024, the `ibm_torino` quantum computer is the only machine (available for the purposes of this thesis) that uses the `Heron` processor, so `ibm_torino` is chosen as the target backend to get the list of natively supported gates from:

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Create a QiskitRuntimeService instance with 'ibm_quantum'
# chosen as the channel (the other possibility is 'ibm_cloud',
# but in this repository, IBM Quantum is used):

# Save your IBM Quantum API token first using the
# QiskitRuntimeService.save_account() method or
# specify your token directly in the QiskitRuntimeService()
# constructor by passing your token in the "token" parameter.
service = QiskitRuntimeService(channel="ibm_quantum")

# As of May 2024, the 'ibm_torino' computer is the only
# machine available to me that uses the 'Heron' processor,
# so 'ibm_torino' is chosen as the target backend:
backend = service.backend("ibm_torino")

# Save the list of supported gates:
supported_instructions = backend.supported_instructions

Show the retrieved list of supported gates along with some basic information about the backend:

In [3]:
from datetime import datetime

# Get the current date and save it in a readable format:
current_date = datetime.now().strftime("%B %d, %Y")

# Show some basic information about the 'ibm_torino'
# computer (also print the current date, because the
# retrieved information may become outdated over time):
print(
    f'The "ibm_torino" backend information (this information was retrieved on {current_date}):\n'
    f"Backend name: {backend.name}\n"
    f"Number of qubits: {backend.num_qubits}\n"
    f"Version of Backend class: {backend.version}\n"
    f"The date that the device went online: {backend.online_date}\n"
    f"Instructions supported by the backend:\n{backend.supported_instructions}"
)

The "ibm_torino" backend information (this information was retrieved on May 10, 2024):
Backend name: ibm_torino
Number of qubits: 133
Version of Backend class: 2
The date that the device went online: 2023-11-02 04:00:00+00:00
Instructions supported by the backend:
['cz', 'id', 'delay', 'measure', 'reset', 'rz', 'sx', 'x', 'if_else', 'for_loop', 'switch_case']


In the next cell, a function is defined that generates input vectors, encodes them, and shows the resulting qubit counts and gate counts in a table. More specifically, the function:

- Takes as input a list of input vector lengths. It then iterates over this list and generates input vectors of the given lengths. The generated vectors only contain `1` as elements. That is because the basis encoding method only allows binary vectors as inputs. For each `1` it uses an `x` gate, while it uses no gate for `0`. So, having all elements as `1` means the basis encoding method uses the maximum number of gates equal to the length of the input vector. As for the other encoding methods, encoding binary vectors does not utilize their full potential because they can not only encode `0` or `1` but also any real number in the interval $[-1, 1]$. However, the point of this Notebook is to use the implemented encoding methods and compare their complexity, not to use them to encode data in the most efficient way possible. As with the basis encoding method, an input vector only containing `1` as its elements requires the maximum number of gates also in all other encoding methods (because the vector containing only `1` as its elements needs to be normalized in order for it to be encoded using the angle, amplitude, and divide-and-conquer encoding methods). To summarize, input vectors are generated in a way that requires the encoding methods to use the maximum number of gates.

- After generating a vector of the desired length, it is encoded using the basis, angle, amplitude, and divide-and-conquer encoding methods from the `qsp` package.

- After encoding the input vector, the quantum circuits are either transpiled or not transpiled. If the circuits are transpiled, they are transpiled with either the lowest or the highest possible optimization level.

- After that, the number of qubits and the number of gates are calculated for each circuit.

- Then, a table is displayed that shows the qubit counts and gate counts for each encoding method for each input vector length.

In [4]:
from qiskit import transpile
from pandas import DataFrame
import numpy as np


def show_table(
    vector_lengths: list[int],
    transpile_circuits: bool = True,
    optimization_level: int = 0,
) -> None:
    """
    This function displays a table that compares the complexity
    of the encoding methods implemented in the 'qsp' package.
    The table shows the qubit and gate counts for each encoding
    method for different input vector lengths. More information
    about this function can be found in the Markdown cell above.

    Parameters:
    -----------
    vector_lengths: list[int]
        A list of input vector lengths. It is used to generate
        input vectors of given lengths and encode them.
    transpile_circuits: bool
        A boolean value indicating whether the created circuits that
        encode input vectors should be transpiled. The default is True.
    optimization_level: int
        An integer value indicating the optimization level to
        use when transpiling the circuits. The default is '0',
        which means no optimization is used. '3' is the highest
        possible optimization level.

    Returns:
    --------
    None
    """

    # Create a list for each encoding method to store the qubit
    # counts and gate counts. Each list will contain tuples of the
    # form (qubit_count, gate_count) for each input vector generated,
    # where the input vector lengths are in the 'vector_lengths'
    # parameter. I-th tuple in the list represents the results of
    # the encoded input vector generated with the i-th length in
    # the 'vector_lengths' parameter.
    results_basis = []
    results_angle = []
    results_amplitude = []
    results_divide_and_conquer = []

    # Iterate over the input vector lengths:
    for vector_length in vector_lengths:
        # Generate an input vector of the desired
        # length with only integer '1' as its elements:
        input_vector = [1] * vector_length

        # First encode the input vector using the basis
        # encoding method, get the resulting tuple by calling
        # the 'get_counts_qubits_gates()' function and append
        # it to the corresponding list:
        results_basis.append(
            get_counts_qubits_gates(
                # Encode the input vector using the basis encoding
                # method by creating an instance of the
                # 'BasisEncoding' class from the 'qsp' package:
                qsp.BasisEncoding(input_vector),
                # Pass the parameters indicating whether to transpile
                # the circuit and the desired optimization level:
                transpile_circuits,
                optimization_level,
            )
        )

        # Normalize the input vector so that the angle, amplitude
        # and divide-and-conquer encoding methods can encode it:
        input_vector /= np.linalg.norm(input_vector)

        # Encode the normalized input vector using the other
        # encoding methods the same way as the above:
        results_angle.append(
            get_counts_qubits_gates(
                qsp.AngleEncoding(input_vector),
                transpile_circuits,
                optimization_level,
            )
        )
        results_amplitude.append(
            get_counts_qubits_gates(
                qsp.AmplitudeEncoding(input_vector),
                transpile_circuits,
                optimization_level,
            )
        )
        results_divide_and_conquer.append(
            get_counts_qubits_gates(
                qsp.DivideAndConquerEncoding(input_vector),
                transpile_circuits,
                optimization_level,
            )
        )

    # Create a dictionary to store the data (column names are keys,
    # and columns themselves are values) that is later shown in a
    # table. Each row contains the input vector length and the qubit
    # and gate counts for each encoding method. So the table will
    # contain 9 columns in total, 1 row for each input vector length:
    data = {
        # The first column contains the input vector lengths:
        "Input<br>length": [length for length in vector_lengths],

        # The next 4 columns contain the qubit counts for each
        # encoding method and the respective input vector length.
        # The columns are created by getting all the qubit counts,
        # which are the first elements in the tuples:
        "Qubits<br>Basis": [pair[0] for pair in results_basis],
        "Qubits<br>Angle": [pair[0] for pair in results_angle],
        "Qubits<br>Amplitude": [pair[0] for pair in results_amplitude],
        "Qubits<br>D-a-c": [pair[0] for pair in results_divide_and_conquer],

        # The last 4 columns contain the gate counts for each
        # encoding method and the respective input vector length:
        # The columns are created by getting all the gate counts,
        # which are the second elements in the tuples:
        "Gates<br>Basis": [pair[1] for pair in results_basis],
        "Gates<br>Angle": [pair[1] for pair in results_angle],
        "Gates<br>Amplitude": [pair[1] for pair in results_amplitude],
        "Gates<br>D-a-c": [pair[1] for pair in results_divide_and_conquer],
        
        # A note: the <br> tag creates a line break in HTML tables.
        # This function specifically displays an HTML table.
    }

    # Create a pandas DataFrame from the dictionary:
    data_frame = DataFrame(data)

    # Get all the column names by creating
    # a list from the dictionary keys:
    data_keys = list(data.keys())

    # Get all the names of the columns that contain the qubit counts:
    data_keys_qubits = data_keys[1:5]

    # Get all the names of the columns that contain the gate counts:
    data_keys_gates = data_keys[5:]

    # Create and display a formatted HTML table containing all the data
    # from the 'data_frame' by getting a 'Styler' object of the DataFrame:
    display(
        data_frame.style
        # Two background gradients are added to the table. These gradients
        # make the table colorful. The first background gradient is for
        # the qubit counts, and the second is for the gate counts (which
        # columns should be affected by the background can be specified
        # by passing the desired column names to the 'subset' parameter).
        # The colormap of the first background is chosen to be 'Blues',
        # and the colormap of the second is picked to be 'Reds'. This
        # will visually divide the table into two parts.
        .background_gradient(cmap="Blues", subset=data_keys_qubits)
        .background_gradient(cmap="Reds", subset=data_keys_gates)
    )


def get_counts_qubits_gates(
    encoding: qsp.base.QuantumStatePreparation,
    transpile_circuit: bool,
    optimization_level: int,
) -> tuple[int, int]:
    """
    Returns a tuple containing the number
    of qubits used by the circuit and the
    number of gates used in the circuit.

    Parameters:
    -----------
    encoding: qsp.base.QuantumStatePreparation
        A QuantumStatePreparation object, this object
        is an abstract parent class of each encoding
        method class and contains a quantum circuit
        that encodes some given input vector.
    transpile_circuit: bool
        A boolean value indicating whether the circuit from
        the QuantumStatePreparation object should be transpiled.
    optimization_level: int
        An integer value indicating the optimization level
        to use when transpiling the circuit, where '0' means
        no optimization used during transpilation, and '3'
        is the highest possible optimization level.

    Returns:
    --------
    tuple[int, int]
        A tuple containing the number of qubits used by the
        circuit (the circuit is retrieved from the 'encoding'
        parameter) and the number of gates used in the circuit.
    """

    # Get the quantum circuit from the encoding:
    circuit = encoding.quantum_circuit

    # If the 'transpile_circuit' variable is set to 'True',
    # transpile the circuit with the given optimization level:
    if transpile_circuit is True:
        circuit = transpile(
            circuit,
            # 'supported_instructions' variable is created a few
            # cells above when the set of supported operations
            # from the 'ibm_torino' computer is retrieved:
            basis_gates=supported_instructions,
            optimization_level=optimization_level,
        )

    # Get the number of qubits used by the circuit:
    qubit_count = circuit.width()

    # Get the number of gates of each kind used in the circuit.
    # The 'count_ops()' method returns a sorted dictionary
    # containing the number of operations of each kind,
    # where the keys are the names of the operations and the
    # values are the number of such operations in the circuit:
    operation_counts = circuit.count_ops()

    # Get the total number of gates used in the
    # circuit by summing the dictionary values:
    gate_count = sum(operation_counts.values())

    # Retrun a tuple containing the number of qubits and gates:
    return qubit_count, gate_count

Now, the defined function is ready to be used to generate tables. It is essential to show the qubit and gate counts for differently-sized input vectors. First, small input vectors are used, specifically from the starting length of just one element to the length of 16 elements (creating an even longer table would not be practical). All three gate count calculation methods are performed, meaning:
1. no transpilation
2. transpilation with no optimization
3. transpilation with the highest optimization level

In [5]:
input_vector_lengths = [length for length in range(1, 17)]
print("Input vector lengths:", input_vector_lengths)

print("\nNo transpilation:")
show_table(input_vector_lengths, False)

print("\nTranspilation with no optimization:")
show_table(input_vector_lengths, True, 0)

print("\nTranspilation with the highest optimization level:")
show_table(input_vector_lengths, True, 3)

Input vector lengths: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

No transpilation:


,Inputlength,QubitsBasis,QubitsAngle,QubitsAmplitude,QubitsD-a-c,GatesBasis,GatesAngle,GatesAmplitude,GatesD-a-c
0,1,1,1,1,3,1,1,1,4
1,2,2,2,1,3,2,2,1,4
2,3,3,3,2,3,3,3,7,4
3,4,4,4,2,3,4,4,11,4
4,5,5,5,3,7,5,5,25,11
5,6,6,6,3,7,6,6,31,11
6,7,7,7,3,7,7,7,31,11
7,8,8,8,3,7,8,8,35,11
8,9,9,9,4,15,9,9,136,26
9,10,10,10,4,15,10,10,153,26



Transpilation with no optimization:


,Inputlength,QubitsBasis,QubitsAngle,QubitsAmplitude,QubitsD-a-c,GatesBasis,GatesAngle,GatesAmplitude,GatesD-a-c
0,1,1,1,1,3,1,5,5,84
1,2,2,2,1,3,2,10,5,84
2,3,3,3,2,3,3,15,31,84
3,4,4,4,2,3,4,20,55,84
4,5,5,5,3,7,5,25,397,311
5,6,6,6,3,7,6,30,519,311
6,7,7,7,3,7,7,35,423,311
7,8,8,8,3,7,8,40,543,311
8,9,9,9,4,15,9,45,2034,834
9,10,10,10,4,15,10,50,2269,834



Transpilation with the highest optimization level:


,Inputlength,QubitsBasis,QubitsAngle,QubitsAmplitude,QubitsD-a-c,GatesBasis,GatesAngle,GatesAmplitude,GatesD-a-c
0,1,1,1,1,3,1,2,0,33
1,2,2,2,1,3,2,6,3,37
2,3,3,3,2,3,3,12,15,35
3,4,4,4,2,3,4,16,6,38
4,5,5,5,3,7,5,20,174,136
5,6,6,6,3,7,6,24,230,136
6,7,7,7,3,7,7,28,182,138
7,8,8,8,3,7,8,32,225,139
8,9,9,9,4,15,9,36,827,361
9,10,10,10,4,15,10,40,921,361


The qubit and gate counts scale linearly with the input vector length when using the basis or angle encoding methods. The transpilation process significantly increases the number of gates of the circuits created by the amplitude and divide-and-conquer encoding methods. Regarding these two encoding methods, it may be beneficial to see how the qubit and gate counts scale when using input vectors with even more elements. So, the following tables are created with input vectors with their lengths equal to powers of 2, starting from 1 to 2048:

In [6]:
input_vector_lengths = [2**length for length in range(0, 12)]
print("Input vector lengths:", input_vector_lengths)

print("\nNo transpilation:")
show_table(input_vector_lengths, False)

print("\nTranspilation with no optimization:")
show_table(input_vector_lengths, True, 0)

print("\nTranspilation with the highest optimization level:")
show_table(input_vector_lengths, True, 3)

Input vector lengths: [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]

No transpilation:


,Inputlength,QubitsBasis,QubitsAngle,QubitsAmplitude,QubitsD-a-c,GatesBasis,GatesAngle,GatesAmplitude,GatesD-a-c
0,1,1,1,1,3,1,1,1,4
1,2,2,2,1,3,2,2,1,4
2,4,4,4,2,3,4,4,11,4
3,8,8,8,3,7,8,8,35,11
4,16,16,16,4,15,16,16,163,26
5,32,32,32,5,31,32,32,355,57
6,64,64,64,6,63,64,64,771,120
7,128,128,128,7,127,128,128,1667,247
8,256,256,256,8,255,256,256,3587,502
9,512,512,512,9,511,512,512,7683,1013



Transpilation with no optimization:


,Inputlength,QubitsBasis,QubitsAngle,QubitsAmplitude,QubitsD-a-c,GatesBasis,GatesAngle,GatesAmplitude,GatesD-a-c
0,1,1,1,1,3,1,5,5,84
1,2,2,2,1,3,2,10,5,84
2,4,4,4,2,3,4,20,55,84
3,8,8,8,3,7,8,40,543,311
4,16,16,16,4,15,16,80,2415,834
5,32,32,32,5,31,32,160,6255,1949
6,64,64,64,6,63,64,320,18895,4248
7,128,128,128,7,127,128,640,54095,8915
8,256,256,256,8,255,256,1280,156367,18318
9,512,512,512,9,511,512,2560,424655,37193



Transpilation with the highest optimization level:


,Inputlength,QubitsBasis,QubitsAngle,QubitsAmplitude,QubitsD-a-c,GatesBasis,GatesAngle,GatesAmplitude,GatesD-a-c
0,1,1,1,1,3,1,2,0,33
1,2,2,2,1,3,2,6,3,37
2,4,4,4,2,3,4,16,6,38
3,8,8,8,3,7,8,32,225,139
4,16,16,16,4,15,16,64,975,370
5,32,32,32,5,31,32,128,2686,863
6,64,64,64,6,63,64,256,8767,1879
7,128,128,128,7,127,128,512,25042,3940
8,256,256,256,8,255,256,1024,69877,8094
9,512,512,512,9,511,512,2048,184246,16431
